# 02. Algebraic Diagnosis
Analyze Rank and Condition Number.

In [ ]:
import os
if not os.path.exists('Credit-Risk-Algebraic-ML'):
    !git clone https://github.com/adriangonz-afk/Credit-Risk-Algebraic-ML.git
os.chdir('Credit-Risk-Algebraic-ML')
print(f'✅ Entorno listo en: {os.getcwd()}')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
X = np.load('data/processed/X_matrix.npy')
X_bias = np.c_[np.ones((X.shape[0], 1)), X]
rank = np.linalg.matrix_rank(X_bias)
print(f'Rank: {rank}')
U, S, Vt = np.linalg.svd(X_bias, full_matrices=False)
print(f'Condition Number: {S.max()/S.min():.2e}')
plt.plot(S, 'o-'); plt.yscale('log'); plt.show()